In [1]:
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import numpy as np

def inertia_hollow_cylinder(mass_kg, r_in_m, r_out_m):
    return 0.5 * mass_kg * (r_in_m**2 + r_out_m**2)

def flywheel_energy_at_rpm(I, rpm):
    omega = (2 * math.pi * rpm) / 60
    return 0.5 * I * omega**2

def flywheel_energy(mass_kg, radius_in, radius_out, rpm_max, rpm_min=0):
    I = inertia_hollow_cylinder(mass_kg, radius_in, radius_out)
    E_max = flywheel_energy_at_rpm(I, rpm_max)
    E_min = flywheel_energy_at_rpm(I, rpm_min)
    E_usable = E_max - E_min
    E_usable_Wh = E_usable / 3600
    return {
        "moment_of_inertia_kgm2": I,
        "max_energy_J": E_max,
        "min_energy_J": E_min,
        "usable_energy_J": E_usable,
        "usable_energy_Wh": E_usable_Wh,
        "rpm_max": rpm_max,
        "rpm_min": rpm_min
    }

def plot_energy_curve(I, rpm_max):
    rpms = np.arange(0, rpm_max + 100, 100)
    energies = [flywheel_energy_at_rpm(I, rpm) for rpm in rpms]

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax1.plot(rpms, energies, label='Stored Energy (J)', color='b')
    ax1.set_xlabel('RPM')
    ax1.set_ylabel('Energy (Joules)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(rpms, [e / 3600 for e in energies], 'r--', label='Stored Energy (Wh)')
    ax2.set_ylabel('Energy (Wh)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title(f'Flywheel Energy Storage vs RPM (I={I:.2f} kg·m²)')
    fig.tight_layout()
    plt.grid(True)
    plt.show()

def plot_discharge_with_losses(I, E_usable, load_W, rpm_min=0,
                               tau_bearing=0.1, k_air=1e-6, k_temp=0):
    T_ambient = 25  # °C
    alpha_temp = 1e-5  # °C per J

    total_time_est = E_usable / load_W if load_W > 0 else 0
    dt = max(1, int(total_time_est / 1000)) if total_time_est > 0 else 1

    t_vals = []
    rpm_vals = []
    power_mech_vals = []
    power_loss_vals = []
    efficiency_vals = []
    temp_vals = []

    E = E_usable
    t = 0

    while E > 0:
        omega = math.sqrt(2 * E / I)
        rpm = (60 * omega) / (2 * math.pi)
        if rpm < rpm_min:
            break

        T = T_ambient + alpha_temp * E
        P_bearing = tau_bearing * omega
        P_air = k_air * omega ** 3
        P_temp = k_temp * (T - T_ambient)

        P_losses = P_bearing + P_air + P_temp
        P_total = load_W + P_losses

        efficiency = load_W / P_total if P_total > 0 else 1.0

        t_vals.append(t)
        rpm_vals.append(rpm)
        power_mech_vals.append(P_total)
        power_loss_vals.append(P_losses)
        efficiency_vals.append(efficiency)
        temp_vals.append(T)

        E -= P_total * dt
        t += dt

    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(t_vals, rpm_vals, 'b-', label='RPM')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('RPM', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(t_vals, power_mech_vals, 'r-', label='Total Power Draw (W)')
    ax2.plot(t_vals, power_loss_vals, 'g--', label='Losses (W)')
    ax2.set_ylabel('Power (W)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    ax3 = ax1.twinx()
    ax3.spines.right.set_position(("outward", 60))
    ax3.plot(t_vals, [e * 100 for e in efficiency_vals], 'm-.', label='Efficiency (%)')
    ax3.set_ylabel('Efficiency (%)', color='m')
    ax3.tick_params(axis='y', labelcolor='m')
    ax3.set_ylim(0, 110)

    ax4 = ax1.twinx()
    ax4.spines.right.set_position(("outward", 120))
    ax4.plot(t_vals, temp_vals, 'c:', label='Temp (°C)')
    ax4.set_ylabel('Temp (°C)', color='c')
    ax4.tick_params(axis='y', labelcolor='c')

    fig.legend(loc='upper right')
    plt.title('Flywheel Discharge: RPM, Power, Losses, Efficiency, Temp')
    fig.tight_layout()
    plt.grid(True)
    plt.show()

def required_input_power(rpm, load_W, tau_bearing=0.1, k_air=1e-6):
    omega = (2 * math.pi * rpm) / 60
    P_bearing = tau_bearing * omega
    P_air = k_air * omega ** 3
    P_losses = P_bearing + P_air
    P_input = load_W + P_losses
    efficiency = load_W / P_input if P_input > 0 else 1.0
    return P_input, P_losses, efficiency

def plot_required_input_power(load_W, rpm_max, tau_bearing=0.1, k_air=1e-6):
    rpms = np.arange(0, rpm_max + 100, 100)
    input_powers = []
    losses = []
    efficiencies = []

    for rpm in rpms:
        P_in, P_loss, eff = required_input_power(rpm, load_W, tau_bearing, k_air)
        input_powers.append(P_in)
        losses.append(P_loss)
        efficiencies.append(eff * 100)

    fig, ax1 = plt.subplots(figsize=(9, 6))

    ax1.plot(rpms, input_powers, 'r-', label='Required Input Power (W)')
    ax1.plot(rpms, losses, 'g--', label='Losses (W)')
    ax1.axhline(y=load_W, color='b', linestyle=':', label='Load Power (W)')
    ax1.set_xlabel('RPM')
    ax1.set_ylabel('Power (W)')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.plot(rpms, efficiencies, 'm-.', label='Efficiency (%)')
    ax2.set_ylabel('Efficiency (%)', color='m')
    ax2.tick_params(axis='y', labelcolor='m')
    ax2.set_ylim(0, 110)
    ax2.legend(loc='upper right')

    plt.title('Required Input Power & Efficiency vs RPM')
    plt.tight_layout()
    plt.show()

def interactive_flywheel(
    mass, r_in, r_out, rpm_min, rpm_max, load, tau_bearing, k_air, k_temp
):
    #if r_out <= r_in:
    #    print("Error: Outer radius must be greater than inner radius.")
    #    return
    if rpm_min >= rpm_max:
        print("Error: Max RPM must be greater than Min RPM.")
        return
    if load <= 0:
        print("Error: Load must be greater than zero.")
        return

    result = flywheel_energy(mass, r_in, r_out, rpm_max, rpm_min)

    print(f"Flywheel specs: {mass:.1f} kg, inner radius {r_in:.3f} m, outer radius {r_out:.3f} m")
    print(f"Moment of inertia: {result['moment_of_inertia_kgm2']:.2f} kg·m²")
    print(f"Max energy: {result['max_energy_J']:.2f} J")
    print(f"Min energy: {result['min_energy_J']:.2f} J")
    print(f"Usable energy: {result['usable_energy_J']:.2f} J ({result['usable_energy_Wh']:.2f} Wh)")

    t_sec = result["usable_energy_J"] / load
    t_hr = t_sec / 3600
    print(f"Runtime at {load} W load (ideal, no losses): {t_sec:.2f} seconds (~{t_hr:.2f} hours)")

    plot_energy_curve(result["moment_of_inertia_kgm2"], rpm_max)

    plot_discharge_with_losses(
        result["moment_of_inertia_kgm2"],
        result["usable_energy_J"],
        load,
        rpm_min,
        tau_bearing=tau_bearing,
        k_air=k_air,
        k_temp=k_temp
    )

    plot_required_input_power(
        load,
        rpm_max,
        tau_bearing=tau_bearing,
        k_air=k_air
    )

interact(
    interactive_flywheel,
    mass=FloatSlider(value=100, min=10, max=500, step=1, description='Mass (kg)'),
    r_in=FloatSlider(value=0, min=0, max=0.49, step=0.01, description='Inner radius (m)'),
    r_out=FloatSlider(value=0.5, min=0.01, max=1.0, step=0.01, description='Outer radius (m)'),
    rpm_min=IntSlider(value=1000, min=0, max=6000, step=100, description='Min RPM'),
    rpm_max=IntSlider(value=6000, min=0, max=12000, step=100, description='Max RPM'),
    load=IntSlider(value=1000, min=1, max=5000, step=100, description='Load (W)'),
    tau_bearing=FloatSlider(value=0.1, min=0, max=1.0, step=0.01, description='Bearing torque (Nm)'),
    k_air=FloatSlider(value=1e-6, min=0, max=1e-4, step=1e-7, readout_format='.7f', description='Air drag coeff'),
    k_temp=FloatSlider(value=0.0, min=0, max=5.0, step=0.1, description='Temp loss W/°C')
)


interactive(children=(FloatSlider(value=100.0, description='Mass (kg)', max=500.0, min=10.0, step=1.0), FloatS…

<function __main__.interactive_flywheel(mass, r_in, r_out, rpm_min, rpm_max, load, tau_bearing, k_air, k_temp)>